# WRI Global Power Plant Database Ingestion Pipeline (Step 3)
# Trino Access Controls and Query Execution
Copyright (C) 2021 OS-Climate

This sample shows:
* How data scientists and users can connect via Trino and use simple SQL to retrieve the data they need into a dataframe
* How we leverage Trino as centralized data access layer to manage fine-grained data access controls in a federated manner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas, pandas profiling and pyarrow
%pip install pandas
%pip install -U pandas-profiling[notebook]
%pip install pyarrow

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [1]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc

Define Environment and Execution Variables

In [3]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Connect to Trino and load the data

In [4]:
# Define execution variables for loading to Iceberg / Trino
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
ingest_table = 'gppd'

In [5]:
engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://caldeirav@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


In [6]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""

In [7]:
df = pd.read_sql(sql, engine).convert_dtypes()

In [8]:
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 108 non-null    string 
 1   country_long            108 non-null    string 
 2   name                    108 non-null    string 
 3   gppd_idnr               108 non-null    string 
 4   capacity_mw             108 non-null    Float64
 5   latitude                108 non-null    Float64
 6   longitude               108 non-null    Float64
 7   primary_fuel            108 non-null    string 
 8   other_fuel1             0 non-null      object 
 9   other_fuel2             0 non-null      object 
 10  other_fuel3             0 non-null      object 
 11  commissioning_year      0 non-null      object 
 12  owner                   0 non-null      object 
 13  source                  108 non-null    string 
 14  url                     108 non-null    st

In [9]:
df.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_data_source,value,observation_method,observation_year
0,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.67,65.795,Solar,None,None,...,None,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,<NA>,<NA>,None,18.29,estimated_generation_gwh,2017
1,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.792,Solar,None,None,...,None,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,<NA>,<NA>,None,18.72,estimated_generation_gwh,2017
2,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,None,None,...,None,GEODB,http://globalenergyobservatory.org,GEODB,1009795,2017,None,174.91,estimated_generation_gwh,2017
3,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.717,Hydro,None,None,...,None,GEODB,http://globalenergyobservatory.org,GEODB,1009797,2017,None,350.8,estimated_generation_gwh,2017
4,AFG,Afghanistan,Nangarhar (Darunta) Hydroelectric Power Plant ...,GEODB0040536,11.55,34.4847,70.3633,Hydro,None,None,...,None,GEODB,http://globalenergyobservatory.org,GEODB,1009787,2017,None,46.12,estimated_generation_gwh,2017


Create another connection engine for os-climate-user3 who has restricted access

In [10]:
engine_user3 = osc.attach_trino_engine("TRINO_USER3", verbose=True, catalog=ingest_catalog)

using connect string: trino://os-climate-user3@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


In [11]:
df_user3 = pd.read_sql(sql, engine_user3).convert_dtypes()

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot select from table osc_datacommons_dev.sandbox.gppd", query_id=20220421_100618_00020_67jqs)

In [ ]:
print(df_user3.info(verbose=True))

In [ ]:
print(df_user3.info(verbose=True))

In [ ]:
df_user3.head()